<a href="https://colab.research.google.com/github/bkhalaf/Colab-Notbooks/blob/master/Paraphrasing_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/paraphrase/data'

train_data.csv


In [ ]:
import pandas as pd 
train_data_path = '/content/drive/MyDrive/Colab Notebooks/paraphrase/data/train_data.csv'
data=pd.read_csv(train_data_path)
data

,title,paraphrase_title
0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi..."
1,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...
2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an..."
3,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ..."
4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...
...,...,...
3570,"Naim al-Goud, mayor of Hit, said people from o...","Naim al-Goud, the newly appointed mayor of Hit..."
3571,"During the flight, engineers misjudged the ext...","During the flight, engineers underestimated th..."
3572,The Web site is registered to Parson under his...,The t33kid.com site is registered to Parson at...
3573,"The woman, Mary Kathryn Miller, 55, was arrest...","Mary Kathryn Miller, 55, of 27 Devon Road, Dar..."


In [ ]:
#feed Model : GPT2
df = pd.read_csv(train_data_path)
df['combined'] = '<s>' + df['paraphrase_title'] +  '</s>' + '>>>>' + '<p>' + df['title'] + '</p>'
df['combined'].to_csv('combined.txt',sep = '\n', index = False)


In [ ]:
df.iloc[0]['combined']

'<s>Referring to him as only "the witness", Amrozi accused his brother of deliberately distorting his evidence.</s>>>>><p>Amrozi accused his brother, whom he called "the witness", of deliberately distorting his evidence.</p>'

In [ ]:
!pip install -qq  transformers 

     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 120 kB 77.2 MB/s 
     |████████████████████████████████| 6.6 MB 67.4 MB/s 


In [ ]:
# Fine tune Model : GPT2
import transformers 
from transformers import pipeline, AutoTokenizer, AutoModel
from transformers import AutoModelWithLMHead,AutoConfig,Trainer,AutoTokenizer , TextDataset , DataCollatorForLanguageModeling ,TrainingArguments
import os 
from transformers import Trainer
def modelTrainer(text_path , epochs , model = 'gpt2', batch_size = 8 , cache_dir = 'cache'): 

  model = AutoModelWithLMHead.from_pretrained(model)
  tokenizer = AutoTokenizer.from_pretrained('gpt2')
  data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer , mlm = False)
  train_dataset = TextDataset (
      tokenizer = tokenizer,
      file_path = text_path,
      block_size = 256) 
  training_args = TrainingArguments (
      output_dir = "gpt2_fine_tune/{}".format(os.path.basename(text_path)),
      num_train_epochs = epochs, 
      per_device_train_batch_size = batch_size,
      warmup_steps = 500,
      save_steps = 2000,
      logging_steps = 10)

  trainer = Trainer (
      model = model,
      args = training_args,
      data_collator = data_collator,
      train_dataset = train_dataset)
      #prediction_loss_only = True)
  trainer.train()
  trainer.save_model()
print("Done")

Done


In [ ]:
modelTrainer('/content/combined.txt',5,'gpt2',8,'cache')

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1002: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (214712 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 838
  Num Epochs = 5
  Instantaneous batch size per device = 8
  To

Step,Training Loss
10,3.128600
20,3.138400
30,3.056600
40,2.975500
50,2.917100
60,2.863000
70,2.749700
80,2.698900
90,2.650200
100,2.558000




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to gpt2_fine_tune/combined.txt
Configuration saved in gpt2_fine_tune/combined.txt/config.json
Model weights saved in gpt2_fine_tune/combined.txt/pytorch_model.bin


In [ ]:
#Test Drive
model_path = '/content/gpt2_fine_tune/combined'
from transformers import pipeline, AutoModelWithLMHead, AutoTokenizer
model = AutoModelWithLMHead.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
generator = pipeline('text-generation', model = model , tokenizer = tokenizer)


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1002: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
loading configuration file /content/gpt2_fine_tune/combined/config.json
Model config GPT2Config {
  "_name_or_path": "/content/gpt2_fine_tune/combined",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
 

In [ ]:
# def paraphrase(input_sentance):
#   return generator('<s>'+input_sentance+ '</s>>>>><p>')
# paraphrase("hi, how are you?")


In [ ]:
# #Trim the output
# def trim_paraphrased(paraphrased_sent):
#   return paraphrased_sent[0]['generated_text'].split('</s>>>>>><p>')[1].split('</p>')[0]

# trim_paraphrased(paraphrase("hi, how are you?"))


In [ ]:
def clean_paraphrased(input_sentence):
  p = generator('<s>' + input_sentence + '</s>>>>><p>')
  return p[0]['generated_text'].split('</s>>>>>><p>')[1].split('</p>')[0]

In [ ]:
clean_paraphrased("This is very awesome")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


IndexError: ignored

In [ ]:
p = generator('<s>' + 'This is very awesome' + '</s>>>>><p>')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
p[0]['generated_text']

"<s>This is very awesome</s>>>>><p>And now it's time for the rest of us to get off a plane and travel by bike, he said.</p>\n<s>Shares closed up 21 cents, or 0"